In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
spark = SparkSession.builder.appName("cluster").getOrCreate()
df = spark.read.options(inferSchema=True,header=True).csv("/FileStore/tables/seeds_dataset.csv")
df.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [0]:
assembler = VectorAssembler(inputCols=df.columns,outputCol='features')
final_data = assembler.transform(df).select('features')
final_data.show()

+--------------------+
|            features|
+--------------------+
|[15.26,14.84,0.87...|
|[14.88,14.57,0.88...|
|[14.29,14.09,0.90...|
|[13.84,13.94,0.89...|
|[16.14,14.99,0.90...|
|[14.38,14.21,0.89...|
|[14.69,14.49,0.87...|
|[14.11,14.1,0.891...|
|[16.63,15.46,0.87...|
|[16.44,15.25,0.88...|
|[15.26,14.85,0.86...|
|[14.03,14.16,0.87...|
|[13.89,14.02,0.88...|
|[13.78,14.06,0.87...|
|[13.74,14.05,0.87...|
|[14.59,14.28,0.89...|
|[13.99,13.83,0.91...|
|[15.69,14.75,0.90...|
|[14.7,14.21,0.915...|
|[12.72,13.57,0.86...|
+--------------------+
only showing top 20 rows



In [0]:
scaler = StandardScaler(inputCol="features",outputCol="scaled_features")
scaler_model = scaler.fit(final_data)
scaled_features = scaler_model.transform(final_data)
scaled_features.show()

+--------------------+--------------------+
|            features|     scaled_features|
+--------------------+--------------------+
|[15.26,14.84,0.87...|[5.24452795332028...|
|[14.88,14.57,0.88...|[5.11393027165175...|
|[14.29,14.09,0.90...|[4.91116018695588...|
|[13.84,13.94,0.89...|[4.75650503761158...|
|[16.14,14.99,0.90...|[5.54696468981581...|
|[14.38,14.21,0.89...|[4.94209121682475...|
|[14.69,14.49,0.87...|[5.04863143081749...|
|[14.11,14.1,0.891...|[4.84929812721816...|
|[16.63,15.46,0.87...|[5.71536696354628...|
|[16.44,15.25,0.88...|[5.65006812271202...|
|[15.26,14.85,0.86...|[5.24452795332028...|
|[14.03,14.16,0.87...|[4.82180387844584...|
|[13.89,14.02,0.88...|[4.77368894309428...|
|[13.78,14.06,0.87...|[4.73588435103234...|
|[13.74,14.05,0.87...|[4.72213722664617...|
|[14.59,14.28,0.89...|[5.01426361985209...|
|[13.99,13.83,0.91...|[4.80805675405968...|
|[15.69,14.75,0.90...|[5.39230954047151...|
|[14.7,14.21,0.915...|[5.05206821191403...|
|[12.72,13.57,0.86...|[4.3715855

In [0]:
kmeans = KMeans(featuresCol="scaled_features",k=3)
model = kmeans.fit(scaled_features)

In [0]:
model.clusterCenters()

Out[19]: [array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714,  8.60366812,
         1.82917353, 10.40106154]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
         2.41585013, 12.29286107]),
 array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066,  7.52397236,
         3.1823335 , 10.39801233])]

In [0]:
model.transform(scaled_features).show()

+--------------------+--------------------+----------+
|            features|     scaled_features|prediction|
+--------------------+--------------------+----------+
|[15.26,14.84,0.87...|[5.24452795332028...|         0|
|[14.88,14.57,0.88...|[5.11393027165175...|         0|
|[14.29,14.09,0.90...|[4.91116018695588...|         0|
|[13.84,13.94,0.89...|[4.75650503761158...|         0|
|[16.14,14.99,0.90...|[5.54696468981581...|         0|
|[14.38,14.21,0.89...|[4.94209121682475...|         0|
|[14.69,14.49,0.87...|[5.04863143081749...|         0|
|[14.11,14.1,0.891...|[4.84929812721816...|         0|
|[16.63,15.46,0.87...|[5.71536696354628...|         1|
|[16.44,15.25,0.88...|[5.65006812271202...|         0|
|[15.26,14.85,0.86...|[5.24452795332028...|         0|
|[14.03,14.16,0.87...|[4.82180387844584...|         0|
|[13.89,14.02,0.88...|[4.77368894309428...|         0|
|[13.78,14.06,0.87...|[4.73588435103234...|         0|
|[13.74,14.05,0.87...|[4.72213722664617...|         0|
|[14.59,14